# LLaVA: Large Language and Vision Assistant

![LLAVA.png](https://velog.velcdn.com/images/tm011899/post/e81c2c49-0dbb-42be-8a3f-f39c436a87e1/image.png)

H Liu, C Li, Q Wu, YJ Lee. arXiv:2304.08485, 2023

https://arxiv.org/abs/2304.08485

https://llava-vl.github.io/

LLaVA (Large Language and Vision Assistant) 모델은 언어만을 사용하는 GPT-4를 활용하여 다중 모달 언어-이미지 지시 데이터를 생성하는 새로운 접근 방식을 제시합니다. 이 논문에서는 시각 인코더(CLIP,ViT-L/14)와 대규모 언어 모델(LLaMA)을 연결하여 일반적인 시각 및 언어 이해를 위한 대규모 멀티모달 모델을 개발하는 것을 목표로 합니다